-----
# import libraries needed to analyses

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import spacy

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
!pip install matplotlib.py